# <center>Interactive JSONiq tutorial</center>


This tutorial introduces the JSONiq language, which declaratively manipulates JSON data. It is an interactive notebook that works with Rumble running as a server in the background.

# Installation of Spark, Python and Jupyter

If you are running this notebook in Jupyter already, we assume you have already installed Python, Jupyter as well as probably Spark. If not, the easiest is to use a distribution such as [Anaconda](https://www.anaconda.com/), which installs everything for you on any operating system.

Then, all you need to do is download this notebook locally, then run

```
jupyter notebook JSONiq-tutorial.ipynb
```

in the same directory to open it. 

# Download Rumble and run it as a server

For this notebook to work, we need two more very simple steps to prepare Rumble. You can execute them on your command line (note that you need to open a new command line window, so that jupyter can continue to run meanwhile).

Download rumble with:
```
wget https://github.com/RumbleDB/rumble/releases/download/v1.6.2/spark-rumble-1.6.2.jar 
```

(you can also download it manually by using [this link](https://github.com/RumbleDB/rumble/releases/download/v1.6.2/spark-rumble-1.6.2.jar) in your browser.

Then, you can start a rumble server by simply executing:
```
spark-submit spark-rumble-1.6.2.jar --executor-cores 2 --server yes --port 9090
```

You should leave it running for all the duration of this tutorial. Then, you can interrupt it with Ctrl-C.

In order for JSONiq to run successfully, you need to execute the following cell (without changing it):

In [ ]:
import requests
import json
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def rumble(line, cell=None):
    if cell is None:
        data = line
    else:   
        data = cell

    response = json.loads(requests.post(server, data=data).text)

    if 'warning' in response:
        print(json.dumps(response['warning']))
    if 'values' in response:
        for e in response['values']:
            print(json.dumps(e))
    elif 'error-message' in response:
        return response['error-message']
    else:
        return response  

As well as this one (where you need to change the port from 9090 to another value if you used a different --port parameter):

In [ ]:
server='http://localhost:9090/jsoniq'

Now we are all set!

## JSON

As explained on the [official JSON Web site](http://www.json.org/), JSON is a lightweight data-interchange format designed for humans as well as for computers. It supports as values:
- objects (string-to-value maps)
- arrays (ordered sequences of values)
- strings
- numbers
- booleans (true, false)
- null

JSONiq provides declarative querying and updating capabilities on JSON data.

## Elevator Pitch

JSONiq is based on XQuery, which is a W3C standard (like XML and HTML). XQuery is a very powerful declarative language that originally manipulates XML data, but it turns out that it is also a very good fit for manipulating JSON natively.
JSONiq, since it extends XQuery, is a very powerful general-purpose declarative programming language. Our experience is that, for the same task, you will probably write about 80% less code compared to imperative languages like JavaScript, Python or Ruby. Additionally, you get the benefits of strong type checking without actually having to write type declarations.
Here is an appetizer before we start the tutorial from scratch.


In [ ]:
%%rumble

let $stores :=
[
  { "store number" : 1, "state" : "MA" },
  { "store number" : 2, "state" : "MA" },
  { "store number" : 3, "state" : "CA" },
  { "store number" : 4, "state" : "CA" }
]
let $sales := [
   { "product" : "broiler", "store number" : 1, "quantity" : 20  },
   { "product" : "toaster", "store number" : 2, "quantity" : 100 },
   { "product" : "toaster", "store number" : 2, "quantity" : 50 },
   { "product" : "toaster", "store number" : 3, "quantity" : 50 },
   { "product" : "blender", "store number" : 3, "quantity" : 100 },
   { "product" : "blender", "store number" : 3, "quantity" : 150 },
   { "product" : "socks", "store number" : 1, "quantity" : 500 },
   { "product" : "socks", "store number" : 2, "quantity" : 10 },
   { "product" : "shirt", "store number" : 3, "quantity" : 10 }
]
let $join :=
  for $store in $stores[], $sale in $sales[]
  where $store."store number" = $sale."store number"
  return {
    "nb" : $store."store number",
    "state" : $store.state,
    "sold" : $sale.product
  }
return [$join]



## And here you go

### Actually, you already knew some JSONiq

The first thing you need to know is that a well-formed JSON document is a JSONiq expression as well.
This means that you can copy-and-paste any JSON document into a query. The following are JSONiq queries that are "idempotent" (they just output themselves):

In [ ]:
%%rumble
{ "pi" : 3.14, "sq2" : 1.4 }

In [ ]:
%%rumble
[ 2, 3, 5, 7, 11, 13 ]

In [ ]:
%%rumble
{
      "operations" : [
        { "binary" : [ "and", "or"] },
        { "unary" : ["not"] }
      ],
      "bits" : [
        0, 1
      ]
    }

In [ ]:
%%rumble
[ { "Question" : "Ultimate" }, ["Life", "the universe", "and everything"] ]

This works with objects, arrays (even nested), strings, numbers, booleans, null.

It also works the other way round: if your query outputs an object or an array, you can use it as a JSON document.
JSONiq is a declarative language. This means that you only need to say what you want - the compiler will take care of the how. 

In the above queries, you are basically saying: I want to output this JSON content, and here it is.

## JSONiq basics

### The real JSONiq Hello, World!

Wondering what a hello world program looks like in JSONiq? Here it is:

In [ ]:
%%rumble
"Hello, World!"

Not surprisingly, it outputs the string "Hello, World!".

### Numbers and arithmetic operations

Okay, so, now, you might be thinking: "What is the use of this language if it just outputs what I put in?" Of course, JSONiq can more than that. And still in a declarative way. Here is how it works with numbers:

In [ ]:
%%rumble
2 + 2

In [ ]:
%%rumble
 (38 + 2) div 2 + 11 * 2


(mind the division operator which is the "div" keyword. The slash operator has different semantics).

Like JSON, JSONiq works with decimals and doubles:

In [ ]:
%%rumble
 6.022e23 * 42

### Logical operations

JSONiq supports boolean operations.

In [ ]:
%%rumble
true and false

In [ ]:
%%rumble
(true or false) and (false or true)

The unary not is also available:

In [ ]:
%%rumble
not true

### Strings

JSONiq is capable of manipulating strings as well, using functions:


In [ ]:
%%rumble
concat("Hello ", "Captain ", "Kirk")

In [ ]:
%%rumble
substring("Mister Spock", 8, 5)

JSONiq comes up with a rich string function library out of the box, inherited from its base language. These functions are listed [here](https://www.w3.org/TR/xpath-functions-30/) (actually, you will find many more for numbers, dates, etc).



### Sequences

Until now, we have only been working with single values (an object, an array, a number, a string, a boolean). JSONiq supports sequences of values. You can build a sequence using commas:


In [ ]:
%%rumble
 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [ ]:
%%rumble
1, true, 4.2e1, "Life"

The "to" operator is very convenient, too:

In [ ]:
%%rumble
 (1 to 100)

Some functions even work on sequences:

In [ ]:
%%rumble
sum(1 to 100)

In [ ]:
%%rumble
string-join(("These", "are", "some", "words"), "-")

In [ ]:
%%rumble
count(10 to 20)

In [ ]:
%%rumble
avg(1 to 100)

Unlike arrays, sequences are flat. The sequence (3) is identical to the integer 3, and (1, (2, 3)) is identical to (1, 2, 3).

## A bit more in depth

### Variables

You can bind a sequence of values to a (dollar-prefixed) variable, like so:

In [ ]:
%%rumble
let $x := "Bearing 3 1 4 Mark 5. "
return concat($x, "Engage!")

In [ ]:
%%rumble
let $x := ("Kirk", "Picard", "Sisko")
return string-join($x, " and ")

You can bind as many variables as you want:

In [ ]:
%%rumble
let $x := 1
let $y := $x * 2
let $z := $y + $x
return ($x, $y, $z)

and even reuse the same name to hide formerly declared variables:

In [ ]:
%%rumble
let $x := 1
let $x := $x + 2
let $x := $x + 3
return $x

### Iteration

In a way very similar to let, you can iterate over a sequence of values with the "for" keyword. Instead of binding the entire sequence of the variable, it will bind each value of the sequence in turn to this variable.

In [ ]:
%%rumble
for $i in 1 to 10
return $i * 2

More interestingly, you can combine fors and lets like so:

In [ ]:
%%rumble
let $sequence := 1 to 10
for $value in $sequence
let $square := $value * 2
return $square

and even filter out some values:

In [ ]:
%%rumble
let $sequence := 1 to 10
for $value in $sequence
let $square := $value * 2
where $square < 10
return $square

Note that you can only iterate over sequences, not arrays. To iterate over an array, you can obtain the sequence of its values with the [] operator, like so:


In [ ]:
%%rumble
[1, 2, 3][]

### Conditions

You can make the output depend on a condition with an if-then-else construct:

In [ ]:
%%rumble
for $x in 1 to 10
return if ($x < 5) then $x
                   else -$x

Note that the else clause is required - however, it can be the empty sequence () which is often when you need if only the then clause is relevant to you.

### Composability of Expressions

Now that you know of a couple of elementary JSONiq expressions, you can combine them in more elaborate expressions. For example, you can put any sequence of values in an array:

In [ ]:
%%rumble
[ 1 to 10 ]

Or you can dynamically compute the value of object pairs (or their key):

In [ ]:
%%rumble
{
      "Greeting" : (let $d := "Mister Spock"
                    return concat("Hello, ", $d)),
      "Farewell" : string-join(("Live", "long", "and", "prosper"),
                               " ")
}

You can dynamically generate object singletons (with a single pair):


In [ ]:
%%rumble
{ concat("Integer ", 2) : 2 * 2 }

and then merge lots of them into a new object with the {| |} notation:

In [ ]:
%%rumble
{|
    for $i in 1 to 10
    return { concat("Square of ", $i) : $i * $i }
|}

## JSON Navigation

Up to now, you have learnt how to compose expressions so as to do some computations and to build objects and arrays. It also works the other way round: if you have some JSON data, you can access it and navigate.
All you need to know is: JSONiq views
an array as an ordered list of values,
an object as a set of name/value pairs


### Objects

You can use the dot operator to retrieve the value associated with a key. Quotes are optional, except if the key has special characters such as spaces. It will return the value associated thereto:

In [ ]:
%%rumble
let $person := {
    "first name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return $person."first name"

You can also ask for all keys in an object:

In [ ]:
%%rumble
let $person := {
    "name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return { "keys" : [ keys($person)] }

### Arrays

The [[]] operator retrieves the entry at the given position:

In [ ]:
%%rumble
let $friends := [ "Jim", "Mary", "Jennifer"]
return $friends[[1+1]]

It is also possible to get the size of an array:

In [ ]:
%%rumble
let $person := {
    "name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return { "how many friends" : size($person.friends) }

Finally, the [] operator returns all elements in an array, as a sequence:

In [ ]:
%%rumble
let $person := {
    "name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return $person.friends[]

### Relational Algebra

Do you remember SQL's SELECT FROM WHERE statements? JSONiq inherits selection, projection and join capability from XQuery, too.

In [ ]:
%%rumble
let $stores :=
[
    { "store number" : 1, "state" : "MA" },
    { "store number" : 2, "state" : "MA" },
    { "store number" : 3, "state" : "CA" },
    { "store number" : 4, "state" : "CA" }
]
let $sales := [
    { "product" : "broiler", "store number" : 1, "quantity" : 20  },
    { "product" : "toaster", "store number" : 2, "quantity" : 100 },
    { "product" : "toaster", "store number" : 2, "quantity" : 50 },
    { "product" : "toaster", "store number" : 3, "quantity" : 50 },
    { "product" : "blender", "store number" : 3, "quantity" : 100 },
    { "product" : "blender", "store number" : 3, "quantity" : 150 },
    { "product" : "socks", "store number" : 1, "quantity" : 500 },
    { "product" : "socks", "store number" : 2, "quantity" : 10 },
    { "product" : "shirt", "store number" : 3, "quantity" : 10 }
]
let $join :=
    for $store in $stores[], $sale in $sales[]
    where $store."store number" = $sale."store number"
    return {
        "nb" : $store."store number",
        "state" : $store.state,
        "sold" : $sale.product
    }
return [$join]

### Access external data

Many implementations support collections of (and indices on) JSON objects or arrays. Rumble reads input through json-file(), parquet-file(), json-doc(), text-file() as well as structured-json-doc().

In [ ]:
%%rumble
json-file("put the path to a JSON lines file here")

In [ ]:
%%rumble
json-doc("put the path to a small JSON file with an object spread over multiple lines here")

It is also possible to get JSON content with an HTTP request, or by parsing it from a string. The EXPath http-client module (described in the Zorba documentation)  allows you to make HTTP requests, and the jn:parse-json() function allows you to use the body as an object or an array.

## I want more

JSONiq supports JSON updates. You can declaratively update your JSON data. JSONiq provides updating expressions. The list of updates that is eventually output by your program is then applied to your JSON data.

```
    copy $people := {
      "John" : { "status" : "single" },
      "Mary" : { "status" : "single" } }
    modify (replace value of json $people.John.status with "married",
            replace value of json $people.Mary.status with "married")
    return $people
    -> { "John" : { "status" : "married" }, "Mary" : { "status" : "married" } }
```

Other updates are insertion into an object or an array, replacement of a value in an object or an array, deletion in an object or an array, renaming an object pair, appending to an array. This is documented on jsoniq.org.

Updates are not supported by Rumble yet, but Zorba supports them.

## Even more

JSONiq can do way more that what is presented here. Here are a couple of highlights:
- JSONiq is a strongly typed language, but is smart enough to not bother you with types when unnecessary. It potentially supports static typing as well to detect errors before you even execute your program.
- You can define your own functions and modules.
- JSONiq provides you with loads of available modules shipped with Zorba.
- JSONiq has tons of further features such as switch, typeswitch and try-catch expressions, universal/existential quantifiers, path expressions, filtering expressions, functors, mappings, grouping, windowing.

## More is not enough

- JSONiq supports XML. Yes: you can manipulate JSON and XML with the same language. JSONiq also exists in a different flavour that is a superset of XQuery, a W3C standard. JSONiq extends its data model to support JSON.
- JSONiq supports scripting. If you need to write a full-fledged, side-effecting Web application, scripting is for you.
The complete JSONiq specification is available on (http://www.jsoniq.org/)[http://www.jsoniq.org/docs/JSONiq/webhelp/index.html]